In [1]:
import pandas as pd 
from pulp import *
import re
import numpy as np

# Data Preparation

In [2]:
with open('data_1.txt', 'r') as fp:
    x1 = fp.readlines()

with open('data_2.txt', 'r') as fp:
    x2 = fp.readlines()

x1 = [re.sub('\n','',i) for i in x1]
x2 = [re.sub('\n','',i) for i in x2]

In [3]:
df = pd.DataFrame([tuple(re.split(',',i)) for i in x1])
df.columns = df.iloc[0]
df.drop(df.index[0], inplace=True)
df.reset_index(drop=True, inplace=True)

df_2 = pd.DataFrame([tuple(re.split(',',i)) for i in x2])
df_2.columns = df_2.iloc[0]
df_2.drop(df_2.index[0], inplace=True)
df_2.reset_index(drop=True, inplace=True)

In [4]:
df[' Inventory Cost(100 units)']=df[' Inventory Cost(100 units)'].astype(int)

In [5]:
df

,Plant,Inventory Cost(100 units),Vendor,Capacity,Disruption Probability(1 - µ),TTR of vendor
0,ABX,31,1,1.2,0.0667,1
1,ABN,30,1,1.2,0.0667,1
2,GHY,32,2,0.9,0.1333,1.2
3,KJU,29,3,0.5,0.2000,1.4
4,PFR,30,4,0.5,0.1667,1.6
5,JKI1,30,5,2.4,0.0667,1.8
6,JKI2,33,5,2.4,0.0667,1.8
7,JKI3,31,5,2.4,0.0667,1.8
8,CFG1,500,6,0.2,0.0667,2
9,CFG2,550,7,0.4,0.1667,2.2


In [6]:
df_2

,ItemID,Vendor,Geographic Location,CFG1,CFG2,CFG3,CFG4
0,ABX,1,US,1,1,1,0
1,ABN,1,US,1,1,1,0
2,GHY,2,Mexico,1,1,2,1
3,KJU,3,Mexico,1,0,1,0
4,PFR,4,US,1,0,1,0
5,JKI1,5,Mexico,0,3,0,0
6,JKI2,5,Mexico,3,0,0,0
7,JKI3,5,Mexico,2,0,3,2
8,CFG1,6,US,1,0,0,0
9,CFG2,7,US,0,1,0,0


In [7]:
d = {}
for i in range(8):
    item = df_2['ItemID'].iloc[i]
    cfg1 = df_2[' CFG1'].iloc[i] 
    cfg2 = df_2[' CFG2'].iloc[i]
    cfg3 = df_2[' CFG3'].iloc[i]
    cfg4 = df_2[' CFG4'].iloc[i]
    d[(item,'CFG1')] = int(cfg1)
    d[(item,'CFG2')] = int(cfg2)
    d[(item,'CFG3')] = int(cfg3)
    d[(item,'CFG4')] = int(cfg4)
B = d

# Defining Parameters

In [170]:
# Create a list of the Plant Nodes
M = df['Plant'].tolist()
M1 = df['Plant'].tolist()[0:8]
M2 = df['Plant'].tolist()[8:]
N = [f'C{i}' for i in range(10)]

# Create a dictionary of capacity for all Plant nodes
capacity = dict(zip(M1,df[' Capacity']))

# Create a dictionary of disruption probability for all Plant nodes
dis_prob = dict(zip(M2,df[' Disruption Probability(1 - µ)']))

# Create a dictionary of inventory costs
inv_cost_x = dict(zip(M1,df[' Inventory Cost(100 units)'].iloc[0:8]))
inv_cost_y = dict(zip(M2,df[' Inventory Cost(100 units)'].iloc[8:]))


# Demand from all customers for different products
demand = {
    'C0':{'CFG1':20, 'CFG2':10, 'CFG3':0, 'CFG4':0},
    'C1':{'CFG1':20, 'CFG2':0, 'CFG3':20, 'CFG4':40},
    'C2':{'CFG1':10, 'CFG2':50, 'CFG3':0, 'CFG4':0},
    'C3':{'CFG1':10, 'CFG2':10, 'CFG3':10, 'CFG4':0},
    'C4':{'CFG1':60, 'CFG2':0, 'CFG3':20, 'CFG4':20},
    'C5':{'CFG1':40, 'CFG2':0, 'CFG3':0, 'CFG4':30},
    'C6':{'CFG1':40, 'CFG2':10, 'CFG3':20, 'CFG4':10},
    'C7':{'CFG1':20, 'CFG2':0, 'CFG3':10, 'CFG4':0},
    'C8':{'CFG1':10, 'CFG2':20, 'CFG3':10, 'CFG4':0},
    'C9':{'CFG1':30, 'CFG2':0, 'CFG3':10, 'CFG4':20},
    }

# Lost Sales Penalty
f = {
    'C0':2.7, 'C1':1.7, 'C2':2.3, 'C3':1.5, 
    'C4':2.7, 'C5':1.7, 'C6':2.3, 'C7':1.5, 
    'C8':2.7, 'C9':1.7
    }

# Defining Random Variables

In [171]:
np.random.seed(9)
v = np.random.randint(0,2,10)
v

array([0, 0, 0, 1, 0, 0, 1, 0, 1, 1])

In [172]:
TR = -1

# Problem Definition

In [242]:
obj = LpProblem("Disruption_Risk_Mitigation",LpMinimize)

# Defining Decision Variables

In [243]:
# Since x is Continuous
x  = {(i,j):
           LpVariable(cat=LpContinuous, 
               lowBound=0, 
               name="x_{0}_{1}".format(i,j)) 
for i in M1 for j in M2}


In [244]:
# Since x is Continuous
y = {(j,k):
           LpVariable(cat=LpContinuous, 
               lowBound=0, 
               name="y_{0}_{1}".format(j,k)) 
for j in M2 for k in N}

In [245]:
# Since x is Continuous
l = {(j,k):
           LpVariable(cat=LpContinuous, 
               lowBound=0, 
               name="l_{0}_{1}".format(j,k)) 
for j in M2 for k in N}

In [246]:
# # This variable indicates lost sales
# l = LpVariable.dict("lost_sales", N, lowBound=0, cat='Continuous')

In [247]:
# Since x is Continuous
X  = {(i,j):
           LpVariable(cat=LpInteger, 
               lowBound=0, upBound=1, 
               name="X_{0}_{1}".format(i,j)) 
for i in M1 for j in M2}


In [248]:
r_x = LpVariable.dicts("inventory_prod_x", M1, lowBound=0, upBound=100, cat=LpContinuous)
r_y = LpVariable.dicts("inventory_prod_y", M2, lowBound=0, upBound=100, cat=LpContinuous)

# Defining Objective

In [271]:
obj += lpSum([f[k]*l[(j,k)]*100 for j in M2 for k in N] +                   # Overall Lost Sales Cost
             [inv_cost_x[i]*r_x[i]*0.01 for i in M1] +                      # Inventory Cost @ Stage 1
             [inv_cost_y[i]*r_y[i] for i in M2] +                           # Inventory Cost @ Stage 2
             [inv_cost_x[i]*x[(i,j)] for i in M1 for j in M2] +             # Processing Cost @ Stage 1
             [inv_cost_y[j]*y[(j,k)]*0.01 for j in M2 for k in N]           # Processing Cost @ Stage 2
             )

c:\programs files\python\python36\lib\site-packages\pulp\pulp.py:1380: UserWarning: Overwriting previously set objective.
  warnings.warn("Overwriting previously set objective.")


# Defining Constraints

In [272]:
# # Capacity Constraint of x
# for i in M1:
#     obj += lpSum(x[(i,j)] for j in M2) >= capacity[i]

In [273]:
# Constraint #1
for j in M2:
    obj += lpSum(x[(i,j)] for i in M1) >= lpSum(y[(j,k)] for k in N)

In [274]:
# Constraint #2
for k in N:
    for j in M2:
        obj += (demand[k][j]) <= l[(j,k)] + (y[(j,k)])

In [275]:
# Constraint #3
for i in M1:
    obj += lpSum(x[(i,j)] for j in M2) - lpSum(y[(j,k)] for k in N for j in M2) <= r_x[i]

In [285]:
# Constraint #4
for j in M2:
    obj += lpSum(y[(j,k)] for k in N) - lpSum(demand[k][j] for k in N) <= r_y[j]

In [286]:
obj.solve()

1

In [287]:
for v in obj.variables():
    if v.varValue>0:
        print(v.name, "=", v.varValue)

x_KJU_CFG1 = 260.0
x_KJU_CFG2 = 100.0
x_KJU_CFG3 = 100.0
x_KJU_CFG4 = 120.0
y_CFG1_C0 = 20.0
y_CFG1_C1 = 20.0
y_CFG1_C2 = 10.0
y_CFG1_C3 = 10.0
y_CFG1_C4 = 60.0
y_CFG1_C5 = 40.0
y_CFG1_C6 = 40.0
y_CFG1_C7 = 20.0
y_CFG1_C8 = 10.0
y_CFG1_C9 = 30.0
y_CFG2_C0 = 10.0
y_CFG2_C2 = 50.0
y_CFG2_C3 = 10.0
y_CFG2_C6 = 10.0
y_CFG2_C8 = 20.0
y_CFG3_C1 = 20.0
y_CFG3_C3 = 10.0
y_CFG3_C4 = 20.0
y_CFG3_C6 = 20.0
y_CFG3_C7 = 10.0
y_CFG3_C8 = 10.0
y_CFG3_C9 = 10.0
y_CFG4_C1 = 40.0
y_CFG4_C4 = 20.0
y_CFG4_C5 = 30.0
y_CFG4_C6 = 10.0
y_CFG4_C9 = 20.0


In [288]:
value(obj.objective)

19876.0

10.0